## 크롤러 만들기

In [1]:
# 크롤러를 만들기 전 필요한 도구들을 임포트합니다.
import requests
import pandas as pd
from bs4 import BeautifulSoup

# 페이지 수, 카테고리, 날짜를 입력값으로 받습니다.
def make_urllist(page_num, code, date): 
  urllist= []
  for i in range(1, page_num + 1):
    url = 'https://news.naver.com/main/list.nhn?mode=LSD&mid=sec&sid1='+str(code)+'&date='+str(date)+'&page='+str(i)   
    news = requests.get(url)

    # BeautifulSoup의 인스턴스 생성합니다. 파서는 html.parser를 사용합니다.
    soup = BeautifulSoup(news.content, 'html.parser')

    # CASE 1
    news_list = soup.select('.newsflash_body .type06_headline li dl')
    # CASE 2
    news_list.extend(soup.select('.newsflash_body .type06 li dl'))
        
    # 각 뉴스로부터 a 태그인 <a href ='주소'> 에서 '주소'만을 가져옵니다.
    for line in news_list:
        urllist.append(line.a.get('href'))
  return urllist

In [2]:
idx2word = {'101' : '경제', '102' : '사회', '103' : '생활/문화', '105' : 'IT/과학'}

In [3]:
from newspaper import Article

#- 데이터프레임을 생성하는 함수입니다.
def make_data(urllist, code):
  text_list = []
  for url in urllist:
    article = Article(url, language='ko')
    article.download()
    article.parse()
    text_list.append(article.text)

  #- 데이터프레임의 'news' 키 아래 파싱한 텍스트를 밸류로 붙여줍니다.
  df = pd.DataFrame({'news': text_list})

  #- 데이터프레임의 'code' 키 아래 한글 카테고리명을 붙여줍니다.
  df['code'] = idx2word[str(code)]
  return df

## 데이터 수집 및 전처리

### 데이터 수집

In [4]:
code_list = [101, 102, 103, 105]

code_list

[101, 102, 103, 105]

In [5]:
def make_total_data(page_num, code_list, date):
  df = None

  for code in code_list:
    url_list = make_urllist(page_num, code, date)
    df_temp = make_data(url_list, code)
    print(str(code)+'번 코드에 대한 데이터를 만들었습니다.')

    if df is not None:
      df = pd.concat([df, df_temp])
    else:
      df = df_temp

  return df

In [32]:
df = make_total_data(4, code_list, 20200520)

101번 코드에 대한 데이터를 만들었습니다.
102번 코드에 대한 데이터를 만들었습니다.
103번 코드에 대한 데이터를 만들었습니다.
105번 코드에 대한 데이터를 만들었습니다.


In [33]:
## 데이터 저장

import os

csv_path = os.getenv("HOME") + "/workplace/aiffel/Explore/03. news_crawler/news_dataset/news_data9.csv"
df.to_csv(csv_path, index=False)

if os.path.exists(csv_path):
  print('{} File Saved!'.format(csv_path))

/home/aiffel0047/workplace/aiffel/Explore/03. news_crawler/news_dataset/news_data9.csv File Saved!


## 데이터 전처리

### 데이터 정제

In [ ]:
csv_path = os.getenv("HOME") + "/workplace/aiffel/Explore/03. news_crawler/news_data6.csv"
df = pd.read_table(csv_path, sep=',')
df.head()

In [17]:
# 정규 표현식을 이용해서 한글 외의 문자는 전부 제거합니다.
df['news'] = df['news'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
df['news']

0     아이뉴스 류은혁 기자 전사적 자원관리시스템 개발업체인 영림원소프트랩은 지난 일 일반...
1     금융노동조합 연대회의 출범식 금융노조 금융노동조합 연대회의 출범식 금융노조아이뉴스 ...
2     금융그룹과 현대건설기계가 아세안 등 해외시장에서 경쟁력을 높이기 위해 업무협약을 맺...
3     정부가 공공참여형 고밀재건축공공재건축 도입 등으로 서울 도심 공급을 확대한다 공공재...
4     더불어민주당과 정부는  부동산 대책을 발표하기에 앞서 일 오전 국회 의원회관에서 당...
                            ...                        
75    삼성전자 전략 스마트폰 갤럭시 등이 스페인에서 모바일 접근성 인증을 받았다 삼성전자...
76    전자 윙 예상 이미지 전자 윙 예상 이미지  전자신문  전자신문인터넷 무단전재 및 ...
77      전자신문  전자신문인터넷 무단전재 및 재배포 금지데이터가 기업과 기관 성패를 좌...
78    이병길 한국테크놀로지 대표 이병길 한국테크놀로지 대표  전자신문  전자신문인터넷 무...
79      전자신문  전자신문인터넷 무단전재 및 재배포 금지일 서울 광화문 스퀘어에서 한 ...
Name: news, Length: 73, dtype: object

In [22]:
print(df.isnull().sum())

news    0
code    0
dtype: int64


In [18]:
# 중복된 샘플들을 제거합니다.
df.drop_duplicates(subset=['news'], inplace=True)

print('뉴스 기사의 개수: ',len(df))

뉴스 기사의 개수:  73


### 데이터 탐색

In [19]:
print(df.groupby('code').size().reset_index(name = 'count'))

    code  count
0  IT/과학     18
1     경제     19
2     사회     20
3  생활/문화     16


### 토큰화

### 불용어 제거
- 조사